## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import sem

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
study_data_complete = pd.merge(mouse_metadata, study_results, how="left", on=["Mouse ID", "Mouse ID"])

# Display the data table for preview
study_data_complete


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [2]:
# Checking the number of mice.
total_number_of_mice_raw_data = len(study_data_complete["Mouse ID"].unique())
total_number_of_mice_raw_data

249

In [3]:
# what are the unique time points?
unique_time_points = study_data_complete["Timepoint"].unique()
unique_time_points

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45], dtype=int64)

In [4]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
# Optional: Get all the data for the duplicate mouse ID. 
duplicated_mouse_ID_and_timepoint = study_data_complete.loc[study_data_complete.duplicated(["Mouse ID", "Timepoint"])]
duplicated_mouse_ID_and_timepoint

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
909,g989,Propriva,Female,21,26,0,45.000000,0
911,g989,Propriva,Female,21,26,5,47.570392,0
913,g989,Propriva,Female,21,26,10,49.880528,0
915,g989,Propriva,Female,21,26,15,53.442020,0
917,g989,Propriva,Female,21,26,20,54.657650,1


In [5]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
study_data_complete_clean = study_data_complete.drop_duplicates(["Mouse ID", "Timepoint"])
study_data_complete_clean

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [6]:
# Checking the number of mice in the clean DataFrame.
total_number_of_mice_clean_data = len(study_data_complete_clean["Mouse ID"].unique())
total_number_of_mice_clean_data

249

## Summary Statistics

In [7]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# The sem() method calculates the standard error of the mean for each column

# This method is the most straighforward, creating multiple series and putting them all together at the end.


In [8]:
#trial a groupby?

# remove un-necessatry columns
summary_stats_mean = study_data_complete_clean[["Drug Regimen", "Tumor Volume (mm3)"]]

# group dataframe by Drug Regimen
summary_stats_mean_grouped = summary_stats_mean.groupby(["Drug Regimen"], as_index=False)

# step one, a mean total for each Drug Regimen
summary_stats_mean_grouped = summary_stats_mean_grouped.mean()

summary_stats_mean_grouped.rename(columns={"Tumor Volume (mm3)": "Tumor Volume Mean (mm3)"}, inplace=True)

summary_stats_mean_grouped.round(2)

,Drug Regimen,Tumor Volume Mean (mm3)
0,Capomulin,40.68
1,Ceftamin,52.59
2,Infubinol,52.88
3,Ketapril,55.24
4,Naftisol,54.33
5,Placebo,54.03
6,Propriva,52.39
7,Ramicane,40.22
8,Stelasyn,54.23
9,Zoniferol,53.24


In [9]:
# remove un-necessatry columns
summary_stats_median = study_data_complete_clean[["Drug Regimen", "Tumor Volume (mm3)"]]

# group dataframe by Drug Regimen
summary_stats_median_grouped = summary_stats_median.groupby(["Drug Regimen"], as_index=False)

# step one, a mean total for each Drug Regimen
summary_stats_median_grouped = summary_stats_median_grouped.median()

summary_stats_median_grouped.rename(columns={"Tumor Volume (mm3)": "Tumor Volume Median (mm3)"}, inplace=True)

summary_stats_median_grouped.round(2)

,Drug Regimen,Tumor Volume Median (mm3)
0,Capomulin,41.56
1,Ceftamin,51.78
2,Infubinol,51.82
3,Ketapril,53.70
4,Naftisol,52.51
5,Placebo,52.29
6,Propriva,50.91
7,Ramicane,40.67
8,Stelasyn,52.43
9,Zoniferol,51.82


In [10]:
# remove un-necessatry columns
summary_stats_variance = study_data_complete_clean[["Drug Regimen", "Tumor Volume (mm3)"]]

# group dataframe by Drug Regimen
summary_stats_variance_grouped = summary_stats_variance.groupby(["Drug Regimen"], as_index=False)

# step one, a variance total for each Drug Regimen
summary_stats_variance_grouped = summary_stats_variance_grouped.var()

summary_stats_variance_grouped.rename(columns={"Tumor Volume (mm3)": "Tumor Volume variance (mm3)"}, inplace=True)

summary_stats_variance_grouped.round(2)

,Drug Regimen,Tumor Volume variance (mm3)
0,Capomulin,24.95
1,Ceftamin,39.29
2,Infubinol,43.13
3,Ketapril,68.55
4,Naftisol,66.17
5,Placebo,61.17
6,Propriva,43.14
7,Ramicane,23.49
8,Stelasyn,59.45
9,Zoniferol,48.53


In [28]:
# combine all the working dataframes
summary_statistics = pd.merge(summary_stats_mean_grouped, summary_stats_median_grouped, how="left", on=["Drug Regimen", "Drug Regimen"])
summary_statistics = pd.merge(summary_statistics, summary_stats_variance_grouped, how="left", on=["Drug Regimen", "Drug Regimen"])
summary_statistics.round(2)

,Drug Regimen,Tumor Volume Mean (mm3),Tumor Volume Median (mm3),Tumor Volume variance (mm3)
0,Capomulin,40.68,41.56,24.95
1,Ceftamin,52.59,51.78,39.29
2,Infubinol,52.88,51.82,43.13
3,Ketapril,55.24,53.70,68.55
4,Naftisol,54.33,52.51,66.17
5,Placebo,54.03,52.29,61.17
6,Propriva,52.39,50.91,43.14
7,Ramicane,40.22,40.67,23.49
8,Stelasyn,54.23,52.43,59.45
9,Zoniferol,53.24,51.82,48.53


In [11]:
# # SECTION NOT WORKING

# # remove un-necessatry columns
# summary_stats_standard_deviation = study_data_complete_clean[["Drug Regimen", "Tumor Volume (mm3)"]]

# summary_stats_standard_deviation = summary_stats_standard_deviation["Tumor Volume (mm3)"].astype(float)

# # group dataframe by Drug Regimen
# summary_stats_standard_deviation_grouped = summary_stats_standard_deviation.groupby(["Drug Regimen"], as_index=False)

# # step one, a standard_deviation total for each Drug Regimen
# # summary_stats_standard_deviation_grouped = summary_stats_standard_deviation_grouped.std()
# summary_stats_standard_deviation_grouped = summary_stats_standard_deviation.agg('std')

# summary_stats_standard_deviation_grouped.rename(columns={"Tumor Volume (mm3)": "Tumor Volume Standard Deviation (mm3)"}, inplace=True)

# summary_stats_standard_deviation_grouped.round(2)

In [12]:
# # Calculate standard error on means
# sems = [sem(s.MEDV) for s in samples]

# # Calculate the range of SEM values
# print(f"The range of SEM values in the sample set is {max(sems)-min(sems)}")

# # Generate a new 30 vehicle sample and calculate the SEM of the sample
# sample = fuel_economy.sample(30)
# print(f"The SEM value for the sample fuel economy data is {sem(sample.Combined_LP100km)}")

# The sem() method calculates the standard error of the mean for each column

# remove un-necessatry columns
summary_stats_mean = study_data_complete_clean[["Drug Regimen", "Tumor Volume (mm3)"]]

# group dataframe by Drug Regimen
summary_stats_mean_grouped = summary_stats_mean.groupby(["Drug Regimen"], as_index=False)

# step one, a mean total for each Drug Regimen
summary_stats_mean_grouped = summary_stats_mean_grouped.mean()

summary_stats_mean_grouped.rename(columns={"Tumor Volume (mm3)": "Tumor Volume Mean (mm3)"}, inplace=True)

summary_stats_mean_grouped.round(2)

,Drug Regimen,Tumor Volume Mean (mm3)
0,Capomulin,40.68
1,Ceftamin,52.59
2,Infubinol,52.88
3,Ketapril,55.24
4,Naftisol,54.33
5,Placebo,54.03
6,Propriva,52.39
7,Ramicane,40.22
8,Stelasyn,54.23
9,Zoniferol,53.24


In [13]:
# what are the unique Drug Regimens?
unique_drug_regimen = study_data_complete["Drug Regimen"].unique()
unique_drug_regimen

array(['Ramicane', 'Capomulin', 'Infubinol', 'Placebo', 'Ceftamin',
       'Stelasyn', 'Zoniferol', 'Ketapril', 'Propriva', 'Naftisol'],
      dtype=object)

In [14]:

# print(study_data_complete_clean["Tumor Volume (mm3)"].mean())
# print(study_data_complete_clean["Tumor Volume (mm3)"].median())

In [15]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method produces everything in a single groupby function


## Bar and Pie Charts

In [16]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 



In [17]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.



In [18]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [19]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [20]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [21]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [22]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
# https://matplotlib.org/stable/gallery/pyplots/boxplot_demo_pyplot.html


## Line and Scatter Plots

In [23]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [24]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [25]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
